In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import time
from scipy import ndimage
import gc
from memory_profiler import memory_usage

### Read markers and relief

In [ ]:
path_in = r"D:\relief"
path_in_markers = r"D:\markers"

In [ ]:
files = tuple(filter(lambda x: ".png" in x, os.listdir(path_in)))
files_markers = tuple(filter(lambda x: ".png" in x, os.listdir(path_in_markers)))
assert len(files) == len(files_markers)

In [ ]:
first = np.array(cv2.imread(os.path.join(path_in, files[0]), cv2.IMREAD_ANYDEPTH))
first_mar = cv2.imread(os.path.join(path_in_markers, files_markers[0]), cv2.IMREAD_ANYDEPTH).astype(np.uint16)
relief = np.empty((len(files), *first.shape), first.dtype)
markers = np.empty((len(files_markers), *first_mar.shape), first_mar.dtype)
relief[0,...] = first
markers[0,...] = first_mar
for z in range(1, len(files)):
    relief[z,...] = cv2.imread(os.path.join(path_in, files[z]), cv2.IMREAD_ANYDEPTH)
    markers[z,...] = cv2.imread(os.path.join(path_in_markers, files_markers[z]), cv2.IMREAD_ANYDEPTH).astype(np.uint16)

In [ ]:
markers.dtype, relief.dtype, markers.shape, relief.shape

### Run via IPSDK

In [ ]:
import PyIPSDK.IPSDKIPLAdvancedMorphology as advmorpho
import PyIPSDK

In [ ]:
ex_mode = PyIPSDK.eWatershedProcessingMode.eWPM_Repeatable

In [ ]:
im3d_relief = PyIPSDK.fromArray(relief)
im3d_markers = PyIPSDK.fromLabelArray(markers)

In [ ]:
start_t = time.perf_counter()

outImg_r = advmorpho.seededWatershed3dImg(im3d_relief, im3d_markers,
                                          PyIPSDK.eWatershedOutputMode.eWOM_Basins, 
                                          ex_mode)

process_time = (time.perf_counter() - start_t) 
print("process_time = {} s".format(process_time))

In [ ]:
ex_mode = PyIPSDK.eWatershedProcessingMode.eWPM_OptimizeSpeed 

In [ ]:
start_t = time.perf_counter()

outImg_r2 = advmorpho.seededWatershed3dImg(im3d_relief, im3d_markers,
                                          PyIPSDK.eWatershedOutputMode.eWOM_Basins, 
                                          ex_mode)

process_time = (time.perf_counter() - start_t) 
print("process_time = {} s".format(process_time))

### Run ITK

In [ ]:
import itk

In [ ]:
NEED_WL = True
itk_image_view_relief = itk.GetImageViewFromArray(relief)
itk_image_view_markers = itk.GetImageViewFromArray(markers)
output = np.zeros_like(markers)
itk_image_view_output = itk.GetImageViewFromArray(output)

In [ ]:
start_t = time.perf_counter()

f = itk.MorphologicalWatershedFromMarkersImageFilter[itk_image_view_relief, 
                                                     itk_image_view_markers].New()
f.SetMarkWatershedLine(NEED_WL)
f.SetFullyConnected(True)
f.SetInput1(itk_image_view_relief)
f.SetInput2(itk_image_view_markers)
labeloutput = f.GetOutput()
labeloutput.SetPixelContainer(itk_image_view_output.GetPixelContainer())

f.Update()

process_time = (time.perf_counter() - start_t) 
print("process_time = {} s".format(process_time))

### Run SMIL

In [ ]:
from smilPython import Image, watershed, CubeSE, SquSE, basins, RhombicuboctahedronSE 

In [ ]:
NEED_WL = True
im_relief = Image("UINT16", *relief.shape)
im_markers = Image("UINT16", *markers.shape)
np_im_relief = im_relief.getNumArray()
np_im_markers = im_markers.getNumArray()
np_im_relief[...] = relief[...]
np_im_markers[...] = markers[...]
imBS = Image(im_markers, "UINT16")
if NEED_WL:
    imWS = Image(imBS, "UINT16")

In [ ]:
start_t = time.perf_counter()

if NEED_WL:
    res = watershed(im_relief, im_markers, imWS, imBS, CubeSE())
else:
    res = basins(im_relief, im_markers, imBS, CubeSE())
    
process_time = (time.perf_counter() - start_t) 
print("process_time = {} s".format(process_time))

In [ ]:
if NEED_WL:
    np_imWS = imWS.getNumArray()
np_imBS = imBS.getNumArray()

### Run Mahotas

In [ ]:
import mahotas

In [ ]:
NEED_WL = True

In [ ]:
start_t = time.perf_counter()

if NEED_WL:
    labels, wl = mahotas.cwatershed(relief, markers, Bc=ndimage.generate_binary_structure(3, 3), return_lines=NEED_WL)
else: 
    labels = mahotas.cwatershed(relief, markers, Bc=ndimage.generate_binary_structure(3, 3), return_lines=NEED_WL)
    
process_time = (time.perf_counter() - start_t) 
print("process_time = {} s".format(process_time))

### Run Skimage

In [ ]:
from skimage import segmentation

In [ ]:
relief_double = relief.astype(np.float64)
NEED_WL = True
MASK_AVAILABLE = False

In [ ]:
if MASK_AVAILABLE:
    path_in_mask = r"D:\mask"
    files_mask = tuple(filter(lambda x: ".png" in x, os.listdir(path_in_mask)))

    first_mask = cv2.imread(os.path.join(path_in_mask, files_mask[0]), cv2.IMREAD_ANYDEPTH).astype(bool)
    mask = np.empty((len(files_mask), *first_mask.shape), bool)
    mask[0,...] = first_mask
    for z in range(1, len(files_mask)):
        mask[z,...] = cv2.imread(os.path.join(path_in_mask, files_mask[z]), cv2.IMREAD_ANYDEPTH).astype(bool)

In [ ]:
start_t = time.perf_counter()

if if MASK_AVAILABLE:
    labels = segmentation.watershed(relief_double, markers,
                                    connectivity=ndimage.generate_binary_structure(3, 3),
                                    watershed_line=NEED_WL,
                                    mask=mask
                                   )
else:
    labels = segmentation.watershed(relief_double, markers,
                                    connectivity=ndimage.generate_binary_structure(3, 3),
                                    watershed_line=NEED_WL
                                   )

process_time = (time.perf_counter() - start_t) 
print("process_time = {} s".format(process_time))

### Run Mamba

In [ ]:
from mamba3D import image3DMb, CUBIC, copyBytePlane3D, watershedSegment3D, basinSegment3D, computeDistance3D

In [ ]:
def fillImageWithArray_3D(array, imOut, cval=0):
    # Checking depth 
    if ((imOut.getDepth()==8 and array.dtype != np.uint8) or \
            (imOut.getDepth()==32 and array.dtype != np.uint32)):
        raiseExceptionOnError()

    # image size
    (wi,hi,di) = imOut.getSize()
    # array size
    (da,ha,wa) = array.shape
        
    # Checking the sizes
    #if wa!=wi or ha!=hi:
    #    raiseExceptionOnError()
    
    pad_w = wi - wa
    pad_h = hi - ha
    pad_d = di - da
    
    data = np.pad(array, ((0, pad_d), (0, pad_h), (0, pad_w)), mode='constant', constant_values=cval)
    print(data.shape)
    #data = data.tostring()
    #chars_err = bytes(np.chararray((data.size * data.itemsize,), 
    #                           1, 
    #                           buffer=data.data, 
    #                          ))
    
    imOut.loadRaw(data.tobytes())

In [ ]:
imDist = image3DMb(*relief.T.shape, 32)
fillImageWithArray_3D(relief.astype(np.uint32), imDist, 2**16-1)
gc.collect()
imMarkers = image3DMb(*markers.T.shape, 32)
fillImageWithArray_3D(markers.astype(np.uint32), imMarkers, 0)
gc.collect()

In [ ]:
NEED_WL = True

start_t = time.perf_counter()

if NEED_WL:
    labels = watershedSegment3D(imDist, imMarkers, grid=CUBIC)
else:
    labels = basinSegment3D(imDist, imMarkers, grid=CUBIC)

process_time = (time.perf_counter() - start_t) 
print("process_time = {} s".format(process_time))

In [ ]:
def getArrayFromImage_3D(imIn, shape):
    if imIn.getDepth()==8:
        dtype = np.uint8
    elif imIn.getDepth()==32:
        dtype = np.uint32
    else:
        raiseExceptionOnError()
            
    (w,h,d) = imIn.getSize()
    # First extracting the raw data out of image imIn
    data = imIn.extractRaw()
    # creating an array with this data
    # At this step this is a one-dimensional array
    array1D = np.frombuffer(data, dtype=dtype)
    # Reshaping it to the dimension of the image
    array3D = array1D.reshape((d,h,w))
    array3D = array3D[0:shape[0], 0:shape[1], 0:shape[2]]
    return array3D

In [ ]:
mamba_labels = getArrayFromImage_3D(imMarkers, markers.shape)